In [1]:
import math
import talib
import numpy as np
from jqdatasdk import *
from talib import MA_Type
from functools import reduce
from jqdatasdk import finance 
import time
import pandas as pd

# 获取近期n天大于某价值的天数
def get_bigger_than_val_counter (close, n, value_list):
    np_close = np.array (close[-n:])
    np_value = np.array (value_list[-n:])
    np_value = np_value * 0.98 # 运行向下浮动2%
    
    diff = np_close - np_value
    return sum (diff > 0)

# 均线
def get_ma (close, timeperiod=5):
    return talib.SMA(close, timeperiod)

# boll
def get_boll (close, timeperiod = 20, deviation = 2):
    ma = get_ma(close, timeperiod)
    #print(ma)
    result = [None] * 2
    upper_bound = [0] * len(close)
    lower_bound = [0] * len(close)
    std = [0] * len(close)
        
    #找到每只股票的boll上线和下线
    for i in range(len(close)):
        distance_sum = 0
        if ma[i] == False:
            pass
        else:
            for j in range(timeperiod):
                distance = (close[i-j] - ma[i]) * (close[i-j] - ma[i])
                distance_sum = distance_sum + distance
        distance_sum = distance_sum / timeperiod
        distance_sum = distance_sum ** 0.5
        std[i] = round(distance_sum, 2 )
        upper_bound[i] = round(ma[i] + deviation * std[i] , 2)
        lower_bound[i] = round(ma[i] - deviation * std[i] , 2)
    
    result[0] = upper_bound
    result[1] = lower_bound
    return result

# 获取均值
def get_avg_price (close, day:int):
    return get_ma (close, day)[-1]
    
# 获取macd技术线
def get_macd (close):
    diff, dea, macd = talib.MACDEXT(close, fastperiod=12, fastmatype=1, slowperiod=26, slowmatype=1, signalperiod=9, signalmatype=1)
    macd = macd * 2
    return diff, dea, macd

# 获取过去n天中的最大价格
def get_max_price (high_values, day):
    return high_values[-day:].max()
        
# 获取最小价格
def get_min_price (low_values, day):
    return low_values[-day:].min()
    
# 单日成交量 大于该股的前5日所有单日成交量的1.5倍
def is_multiple_stocks (volume, days = 5):
    vol = volume[-1]
    
    for i in range(2, days + 2):
        if volume[-i] * 1.5 > vol:
            return False
    
    return True
    
# 获取波动的百分比的标准差
def get_std_percentage(var_list):
    v_array = np.array(var_list)
    
    # 获取每个单元的涨跌幅百分比
    ratio = (v_array - np.mean(v_array))/np.mean(v_array)
    
    # 每个单元的平方，消除负号
    ratio_p2 = ratio * ratio
    
    # 得到平均涨跌幅的平方
    ratio = ratio_p2.sum()/len(v_array)
    
    # 开方 得到平均涨跌幅
    return np.sqrt(ratio)

# 均线多头排列
def get_avg_array (avg_list):
    ratio = 0;
    count = len(avg_list)
    
    for i in range(1, count):
        if avg_list[-i-1] < avg_list[-i]:
            return 0
        ratio += (avg_list[-i-1] - avg_list[-i]) / avg_list[-i]
    
    return ratio

# 给股票评分
#def get_stocks_score (tick_list, data_frame, pass_time):
def get_stocks_score (data_frame, pass_time):
    #score = {}
    #avg_score = {}
    
    #for code in tick_list:
    high = data_frame['high'].values
    low = data_frame['low'].values
    close = data_frame['close'].values
        
    # 获取均线矩阵， 不要5日线
    avg_list = np.array([get_ma(close, 10), get_ma(close, 20),
                        get_ma(close, 30), get_ma(close, 60)])

    box_ratio = 0
    for i in range(2, 32): # 不考虑当日的影响
        box_ratio += get_std_percentage(avg_list.T[-i])
        
    #print (str(code)+" box " + str(box_ratio))
        
    array_ratio = 0
    for i in range (1, 6):
        array_ratio += get_avg_array (avg_list.T[-i])
    #print (str(code)+" array " + str(array_ratio))
        
    #avg_score[code] = array_ratio
    avg_score = array_ratio
    #score[code] = box_ratio - array_ratio
    score = box_ratio - array_ratio
        
    #tick_list.clear()
    #score = sorted(score.items(), key = lambda kv:(kv[1], kv[0]), reverse=False)
    #for key in score:
    #    sec_info = get_security_info (key[0])
    #    print("\t"+str(sec_info.display_name) + ": " +str(key[0]) + " 得分是 " + str(key[1]))
    #    tick_list.append(key[0])
        
    #return tick_list
    return score

def get_wma(close, timeperiod):
    array = np.array(close)
    return talib.WMA(array, timeperiod)

def get_hma(close, timeperiod):
    wma1 = get_wma(close,timeperiod)
    wma2 = get_wma(close,timeperiod/2)
    return get_wma(2*wma2 - wma1, timeperiod ** 0.5)

def get_dema(close, timeperiod):
    array = np.array(close)
    return talib.DEMA(array,timeperiod)

def get_ema(close, timeperiod):
    array = np.array(close)
    return talib.EMA(array,timeperiod)

def get_price_efficiency(close, timeperiod):
    denominator = 0
    for i in range(0,timeperiod):
        denominator = denominator + abs(close[-i-2] - close[-i-1])
    return 100 * abs(close[-1] - close[-timeperiod]) / denominator

def get_quantity_efficiency(volume, timeperiod):
    return (volume[-1] - volume[-timeperiod]) / volume[-timeperiod]

In [2]:
auth('19145630091','Cxt123456')
print(is_auth)
print(get_query_count())

#获取日期
localtime = time.asctime(time.localtime())
#print(localtime)
today = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())[0:10]
enddate = today
stockprice = get_price( '000001.XSHG' , start_date=None , end_date=enddate , fields = ['high', 'low','volume'], count = 2)
print(stockprice)
lastday_volume = stockprice['volume'][0]
yesterday = str(stockprice.loc[(stockprice['volume'] == lastday_volume)].index)[16:26] 

auth success 
<function is_auth at 0x000001BB8CCA9730>
{'total': 1000000, 'spare': 727153}
                 high        low        volume
2021-03-25  3382.2300  3344.9700  2.706761e+10
2021-03-26  3422.4041  3373.3171  2.178681e+10


In [ ]:
#读取经过筛选的股票代码
data = pd.read_table('C:/My/Intern/PF/Stockcode.txt',header=None,encoding='gb2312',delim_whitespace=True)
#print(data.head())
data.columns=['stkcd']

#实验日期 
yesterday = '2021-03-25'

market = get_price('000001.XSHG', count = 110, end_date = yesterday, panel = False, fields = ['close'])
shi = market['close'].values


stocklist = {}

for i in range(len(data.loc[:,'stkcd'])):
    stkcd = data.loc[i,'stkcd']
    try:
        #筛选出ST股票
        #name = get_security_info(stkcd).display_name
        #print(name)
        #if 'ST' in name:
            #continue
                
        df = pd.read_csv('C:/My/Intern/PF/Database/'+stkcd+'.csv')
        close = df['close'].values
        volume = df['volume'].values
        
        ema_close_short = get_ema(close,10)
        ema_close_long = get_ema(close,26)
        trend = [None] * len(ema_close_short)
        for i in range(len(ema_close_short)):
            try:
                trend[i] = ema_close_short[i] / ema_close_long[i]
            except:
                trend[i] = 0
        accelerator = get_ema(trend, 12)
        
        if not ( trend[-1] > 1 and accelerator[-1] > 1):
            continue

        wma_close = get_wma(close,4)
        wma_volume = get_wma(volume,4)
        
        price_efficiency = get_price_efficiency(wma_close, 10)
        quant_effi = get_quantity_efficiency(wma_volume, 10)
        
        if price_efficiency > 50 and quant_effi < 0:
            continue
        
        stocklist[stkcd] = accelerator[-1] + trend[-1]
        
    except:
        print(stkcd)
        pass

0.9851106059217039
0.9789822223813789
1.0165715520090188
1.0052611186763465
0.9749271158604739
0.9775114369367663
1.0240743575357047
1.0562076584519013
1.0100931861132665
1.0109820856542593
1.034447538263827
1.0224673931034998
0.9926783425243029
0.9935461012923379
1.021200306814846
1.011996762502161
1.026529766730293
1.0480732964149924


In [ ]:
stocklist = sorted(stocklist.items(),key = lambda kv:(kv[1],kv[0]),reverse = True)

In [ ]:
print(stocklist)

In [ ]:
print(len(stocklist))